In [1]:
import pandas as pd
import numpy as np
#import tensorflow as tf
from tqdm import tqdm
import feather

In [2]:
from boto.s3.connection import S3Connection
from boto.s3.key import Key
import sys, os
import re
import json
from urllib.request import FancyURLopener
import cv2
import glob
import zipfile
from multiprocessing import Pool

In [3]:
KEY_FILE = '..//..//..//data//aws_keys//keys.txt'

f = open(KEY_FILE,'rb')
key_data = str(f.read())
keys = re.findall(pattern='\=([a-zA-Z0-9]*)',string=key_data)

AWS_KEY = keys[0]
AWS_SECRET = keys[1]

del keys

In [4]:
BUCKET = 'imaterialist-challenge'
PATH_TO_DATA = '..//..//..//data//imaterialist//'

In [5]:
labeldf = feather.read_dataframe(PATH_TO_DATA+'labeldf.feather')
task_df = feather.read_dataframe(PATH_TO_DATA+'taskdf.feather')
train_annotations = feather.read_dataframe(PATH_TO_DATA+'train_annotations.feather')

In [6]:
def readImage(imageId):
    return cv2.imread(PATH_TO_DATA+imageId+'.jpg')

In [7]:
train_annotations.head(5)

,imageId,labelId,taskId
0,1,6,5
1,2,7,6
2,3,8,7
3,4,10,9
4,5,14,12


# Single Core version

In [ ]:
train_images = []
for row in tqdm(train_annotations.iterrows()):
    try:
        appended = [str(row[1]['imageId']),readImage(str(row[1]['imageId']))]
        appended_shape = np.shape(appended[1])
        train_images.append(appended+[appended_shape])
    except:
        x = 1

16681it [11:11, 32.15it/s]

In [26]:
train_images[:10]

[['1', None, ()],
 ['2', None, ()],
 ['3', None, ()],
 ['4', None, ()],
 ['5', None, ()],
 ['5', None, ()],
 ['6', None, ()],
 ['7', None, ()],
 ['8', None, ()],
 ['8', None, ()]]

# Multi-Core version

In [11]:
def GetImageDetails(imageId):
    try:
        appended = [imageId,readImage(imageId)]
        appended_shape = np.shape(appended[1])
        return(appended+[appended_shape])
    except:
        x = 1

In [9]:
train_images = []

In [ ]:
pool = Pool()
pool.map_async(GetImageDetails, list(train_annotations['imageId']), callback = train_images.append )
pool.close()
pool.join()

Exception in thread Thread-13:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.5/multiprocessing/pool.py", line 429, in _handle_results
    task = get()
  File "/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.5/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.5/multiprocessing/connection.py", line 411, in _recv_bytes
    return self._recv(size)
  File "/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.5/multiprocessing/connection.py", line 386, in _recv
    buf.write(chunk)
MemoryError



In [ ]:
len(train_images)